<a href="https://colab.research.google.com/github/enternalsaga/stablediffusion_webui/blob/master/SD_webui_colab_AIDVN_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os
import zipfile
from ipywidgets import widgets
from IPython.display import display

reg_folder = ""

# Define a dictionary of model links
models = {
    "photon": "https://huggingface.co/digiplay/Photon_v1/resolve/main/photon_v1.safetensors",
    "chillout_mix": "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/chillout_mix-pruned.safetensors",
    "sd1.5": "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/stable_diffusion_1_5-pruned.safetensors",
}

# Define a dictionary of regularization links
regs = {
    "none": "",
    "person": "https://huggingface.co/datasets/ProGamerGov/StableDiffusion-v1-5-Regularization-Images/resolve/main/person_v1-5_mse_vae_ddim50_cfg7_n2115.zip",
    "woman": "https://huggingface.co/datasets/ProGamerGov/StableDiffusion-v1-5-Regularization-Images/resolve/main/woman_v1-5_mse_vae_ddim50_cfg7_n4420.zip",
    "interior": "https://huggingface.co/datasets/enternalsaga/SD-1.5_reg_img/resolve/main/regularization-interior.zip",
    "exterior_townhouse": "https://huggingface.co/datasets/enternalsaga/SD-1.5_reg_img/resolve/main/regularization-exterior_house.zip",
}


#@title <font size="5" color="orange">Step 1: Upload images and print folder names </font>
from google.colab import drive
drive.mount('/content/drive')



# construct paths
projectPath = '/content/drive/MyDrive/AIDVN_Train/TrainFolder/' + dataset_name
imagePath = projectPath + '/img/' + str(Number_of_epoches) + '_'+ dataset_name
loraPath = projectPath + '/lora/'

!mkdir -p {loraPath}

import os
from google.colab import files
import shutil

if os.path.exists(imagePath):
  print(f'Error: Folder {imagePath} already exists. Please use a different project folder or dataset names. Skip uploading.\n')
else:
  !mkdir -p {imagePath}
  uploaded = files.upload()
  for filename in uploaded.keys():
      dst_path = imagePath + '/' + filename
      shutil.move(filename, dst_path)
  print('Images uploaded successfully.\n')

# print image paths
print(f"Image folder to caption: {imagePath}")
print(f"Lora Image folder: {projectPath}")
print(f"Lora output folder: {loraPath}")

# Define the folder path for download
checkpoint_folder = '/content/chkp_folder/'
reg_subfolder = os.path.join("/content/reg_folder", reg_folder)
# Create the content/reg/ folder if it doesn't exist
if not os.path.exists(reg_subfolder):
    os.makedirs(reg_subfolder)

# Download and unzip the file from the regularization links
if reg_folder in regs and regs[reg_folder]:
    reg_link = regs[reg_folder]
    zip_filename = reg_link.split('/')[-1]  # Extract the filename from the URL
    zip_path = os.path.join("/content/reg_folder", zip_filename)  # Path to save the zip file
    reg_path = os.path.join(reg_subfolder, "1_" + reg_folder)

    # Check if the folder is empty before extracting
    if not os.listdir(reg_subfolder):
        # Download the zip file
        !wget "$reg_link" -O "$zip_path"

        # Extract the downloaded file directly into the subfolder
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(reg_subfolder)
        # Rename the extracted folder to "1_" + Regularisation_folder
        extracted_folder = os.path.join(reg_subfolder, os.listdir(reg_subfolder)[0])
        new_folder_name = os.path.join(reg_subfolder, "1_" + reg_folder)
        os.rename(extracted_folder, new_folder_name)
        print(f"Renamed folder to {new_folder_name}")

        # Provide a download link for the user
        print(f"Regularization files downloaded and saved to '{reg_subfolder}'.")
    else:
        print(f"The folder '{reg_subfolder}' is not empty. Skipping extraction.")
else:
    print("No regularization link provided. Please specify a regularization link.")

# Check if CheckpointDownloadLink is provided, otherwise use base_model
CheckpointDownloadLink = custom_base_model
if not CheckpointDownloadLink and base_model in models:
    CheckpointDownloadLink = models[base_model]

# Download the file from the link and save it to the specified folder
if CheckpointDownloadLink:
    filename = CheckpointDownloadLink.split('/')[-1]  # Extract the filename from the URL
    output_path = os.path.join(checkpoint_folder, filename)
    !wget "$CheckpointDownloadLink" -O "$filename"
    os.makedirs(checkpoint_folder, exist_ok=True)

    # Move the downloaded file to the content folder
    !mv "$filename" "/content/chkp_folder/$filename"

    # Provide a download link for the user
    print(f"File downloaded and saved to {output_path}. You can now use it")
else:
    print("No download link provided. Please specify a download link.")

#@title Train with Kohya's Stable Diffusion Trainers

def clear():
    from IPython.display import clear_output; return clear_output()

!pip install dadaptation==3.1 diffusers[torch]==0.24.0 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.19.4
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf


# Install bitsandbytes
!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!CUDA_VERSION=117 make -s cuda11x
!python setup.py install


%cd /content
!git clone https://github.com/bmaltais/kohya_ss.git
%cd kohya_ss/
!git checkout v21.8.9

clear()

# Add custom presets

%cd /content/
!git clone https://github.com/enternalsaga/kohya_ss_preset.git
!git pull

import os
import shutil

# Directory to be cleared
folder_path = '/content/kohya_ss/presets/lora/'

# Clear the contents of the folder
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
else:
    print(f"The directory {folder_path} does not exist.")

# Create the target directory if it doesn't exist
os.makedirs('/content/drive/MyDrive/AIDVN_Train/config/', exist_ok=True)

# Copy .json files from cloned repo to the specified folder
!cp /content/kohya_ss_preset/*.json /content/drive/MyDrive/AIDVN_Train/config/

# Create symbolic links
if os.path.exists('/content/drive/MyDrive/AIDVN_Train/config/'):
    for filename in os.listdir('/content/drive/MyDrive/AIDVN_Train/config/'):
        src = os.path.join('/content/drive/MyDrive/AIDVN_Train/config/', filename)
        dst = os.path.join(folder_path, filename)
        if not os.path.exists(dst):
            os.symlink(src, dst)

clear()

# add pwd to python path or else blip captioning won't work
%cd /content/kohya_ss/
%env PYTHONPATH=/env/python:/content/kohya_ss
!python kohya_gui.py --share --headless